In [1]:
import cv2
import os
import glob
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
!pip install shapely
from shapely import Point,Polygon,LineString
from shapely.ops import split
from shapely import to_ragged_array
from tqdm import tqdm


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
img_dir = '/teamspace/studios/this_studio/data/images/'
label_file = '/teamspace/studios/this_studio/data/annotations/train.json'

img_files = os.listdir(img_dir)

imgs_files_df = pd.DataFrame(zip(img_files),columns=['img_name'])

imgs_files_df['image_id'] = imgs_files_df.apply(lambda row: int(row['img_name'].split('xray_')[1].split('.png')[0]), axis=1)

imgs_files_df

,img_name,image_id
0,xray_00609.png,609
1,xray_00610.png,610
2,xray_00611.png,611
3,xray_00612.png,612
4,xray_00613.png,613
...,...,...
1314,xray_01923.png,1923
1315,xray_01924.png,1924
1316,xray_01925.png,1925
1317,xray_01926.png,1926


In [3]:
with open(label_file,'r') as f:
    labels = json.load(f)

In [4]:
img_info_df = pd.DataFrame.from_dict(labels['images'])
img_info_df.rename(columns={'id':'image_id'},inplace=True)

In [5]:
label_target_df = pd.DataFrame.from_dict(labels['annotations'])

In [6]:
label_target_df = pd.merge(label_target_df,imgs_files_df,on='image_id')

In [7]:
label_target_df

,id,image_id,category_id,segmentation,bbox,iscrowd,area,img_name
0,684,609,8,"[[168.47727272727272, 390.90909090909093, 154....","[100.29545454545455, 274.4318181818182, 69.886...",0,2672.0,xray_00609.png
1,685,610,8,"[[289.69767441860466, 363.60465116279073, 267....","[139.69767441860466, 290.3488372093023, 150.0,...",0,3013.0,xray_00610.png
2,686,611,8,"[[273.47727272727275, 321.59090909090907, 264....","[132.0, 201.70454545454547, 141.47727272727275...",0,3519.0,xray_00611.png
3,687,612,8,"[[182.6818181818182, 393.75, 203.1363636363636...","[182.6818181818182, 391.47727272727275, 59.659...",0,466.0,xray_00612.png
4,688,613,8,"[[236.09090909090907, 402.27272727272725, 217....","[155.97727272727272, 269.8863636363636, 80.113...",0,3020.0,xray_00613.png
...,...,...,...,...,...,...,...,...
1327,2011,1923,10,"[[302.96253163179244, 120.66882533808604, 344....","[41.206225680933855, 79.37743190661479, 349.02...",0,10015.0,xray_01923.png
1328,2012,1924,10,"[[201.66433566433568, 264.3356643356643, 229.6...","[35.93006993006993, 213.986013986014, 209.7902...",0,13908.0,xray_01924.png
1329,2013,1925,10,"[[296.4195804195804, 357.34265734265733, 320.1...","[198.5174825174825, 326.5734265734266, 155.244...",0,10869.0,xray_01925.png
1330,2014,1926,10,"[[168.44755244755245, 396.50349650349654, 134....","[65.65034965034965, 386.7132867132867, 150.349...",0,5882.0,xray_01926.png


In [8]:
label_target_df = pd.merge(label_target_df,img_info_df,on='image_id')

In [9]:
label_target_df

,id,image_id,category_id,segmentation,bbox,iscrowd,area,img_name,height,width,file_name
0,684,609,8,"[[168.47727272727272, 390.90909090909093, 154....","[100.29545454545455, 274.4318181818182, 69.886...",0,2672.0,xray_00609.png,448,580,xray_00609.png
1,685,610,8,"[[289.69767441860466, 363.60465116279073, 267....","[139.69767441860466, 290.3488372093023, 150.0,...",0,3013.0,xray_00610.png,448,640,xray_00610.png
2,686,611,8,"[[273.47727272727275, 321.59090909090907, 264....","[132.0, 201.70454545454547, 141.47727272727275...",0,3519.0,xray_00611.png,448,540,xray_00611.png
3,687,612,8,"[[182.6818181818182, 393.75, 203.1363636363636...","[182.6818181818182, 391.47727272727275, 59.659...",0,466.0,xray_00612.png,448,480,xray_00612.png
4,688,613,8,"[[236.09090909090907, 402.27272727272725, 217....","[155.97727272727272, 269.8863636363636, 80.113...",0,3020.0,xray_00613.png,448,580,xray_00613.png
...,...,...,...,...,...,...,...,...,...,...,...
1327,2011,1923,10,"[[302.96253163179244, 120.66882533808604, 344....","[41.206225680933855, 79.37743190661479, 349.02...",0,10015.0,xray_01923.png,448,400,xray_01923.png
1328,2012,1924,10,"[[201.66433566433568, 264.3356643356643, 229.6...","[35.93006993006993, 213.986013986014, 209.7902...",0,13908.0,xray_01924.png,448,580,xray_01924.png
1329,2013,1925,10,"[[296.4195804195804, 357.34265734265733, 320.1...","[198.5174825174825, 326.5734265734266, 155.244...",0,10869.0,xray_01925.png,448,480,xray_01925.png
1330,2014,1926,10,"[[168.44755244755245, 396.50349650349654, 134....","[65.65034965034965, 386.7132867132867, 150.349...",0,5882.0,xray_01926.png,448,480,xray_01926.png


In [10]:
def mod_image(img,source_polygon,destination_polygon):

    # Create a mask for the source polygon
    source_mask = np.zeros_like(img, dtype=np.uint8)
    cv2.fillPoly(source_mask, [source_polygon], (255, 255, 255))

    # Extract the region of interest (ROI) from the source polygon
    source_roi = cv2.bitwise_and(img, source_mask)

    # Inpaint the original source polygon region to backfill it seamlessly
    # The mask for inpainting needs to be a single channel (grayscale) where white denotes the area to be inpainted.
    inpaint_mask = cv2.cvtColor(source_mask, cv2.COLOR_BGR2GRAY)
    inpainted_img = cv2.inpaint(img, inpaint_mask, 12, cv2.INPAINT_TELEA)

    # Calculate the offset for pasting the ROI
    source_x_min, source_y_min = np.min(source_polygon, axis=0)
    source_x_max, source_y_max = np.max(source_polygon, axis=0)

    dest_x_min, dest_y_min = np.min(destination_polygon, axis=0)
    dest_x_max, dest_y_max = np.max(destination_polygon, axis=0)

    x_offset = dest_x_min - source_x_min
    y_offset = dest_y_min - source_y_min

    # Create a translated mask for the ROI at the destination
    translated_roi_mask = np.zeros_like(img, dtype=np.uint8)
    cv2.fillPoly(translated_roi_mask, [destination_polygon], (255, 255, 255))

    roi_width = source_x_max - source_x_min
    roi_height = source_y_max - source_y_min

    extracted_roi = source_roi[source_y_min:source_y_max, source_x_min:source_x_max]

    #print(dest_y_min,roi_height,dest_x_min, roi_width, img.shape)

    # Create a new image for the translated ROI to be pasted on
    pasted_roi_img = np.zeros_like(img, dtype=np.uint8)
    pasted_roi_img[dest_y_min:dest_y_min + roi_height, dest_x_min:dest_x_min + roi_width] = extracted_roi

    cv2.fillPoly(inpainted_img, [destination_polygon], (255, 255, 255))

    translated_roi_mask = translated_roi_mask > 0
    inpainted_img[translated_roi_mask] = pasted_roi_img[translated_roi_mask]

    return inpainted_img

In [11]:
out_frag_img_dir = '/teamspace/studios/this_studio/data/fragmented/'

annotations = [] # "id": 9465, "image_id": 9465, "category_id": 4, "segmentation": [[88.60215053763437, 677.7956989247311]], "bbox": [80.0, 677.7956989247311, 80.64516129032256, 140.86021505376345], "iscrowd": 0, "area": 10283.0}
categories = [] #categories": [{"id": 1, "name": "Baton"}, {"id": 2, "name": "Pliers"}, {"id": 3, "name": "Hammer"}, {"id": 4, "name": "Powerbank"},
images = [] # "images": [{"height": 448, "width": 580, "id": 0, "file_name": "xray_easy00000.png"}
id = 0

max_image_id = max(label_target_df["image_id"]) + 1

nimgs = len(label_target_df)
for i in tqdm(range(nimgs)):

    #print(i)

    multi=True

    path = img_dir + label_target_df['img_name'].values[i]
    polygon = label_target_df['segmentation'].values[i]
    bbox = np.array(label_target_df['bbox'].values[i],np.int32)

    box = (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3])

    width = bbox[2]
    height = bbox[3]

    img = cv2.imread(path)

    pts = np.array([ [polygon[0][2*i],polygon[0][2*i+1]] for i in range( int(len(polygon[0]) /2) -1 )],np.int32)

    # 1. Define your Polygon
    polygon = Polygon( [tuple(x) for x in pts] )

    # 2. Define your Splitting Line
    splitter = LineString([ (int(bbox[0]+width*0.5), int(bbox[1] + height)) , (int(bbox[0]+width*0.5) , int(bbox[1] ))])

    # 3. Use shapely.ops.split
    result = split(polygon, splitter)

    polygons = []

    for geom in result.geoms:
        if isinstance(geom, Polygon):
            xx, yy = geom.exterior.coords.xy
            polygons.append( np.array([ [xx[i],yy[i]] for i in range(len(xx)) ],np.int32)   )

    if len(polygons) != 2:
        multi = False

    if multi:

        pts = pts.reshape((-1, 1, 2)) # Reshape for polylines function
        #cv2.polylines(img, [pts], True, (0, 255, 255), 2) # Yellow color, 2 thickness

        #cv2.polylines(img, [polygons[0]], True, (0, 0, 255), 2) # Yellow color, 2 thickness
        #cv2.polylines(img, [polygons[1]], True, (0, 255, 255), 2) # Yellow color, 2 thickness
        #cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[0]+width, bbox[1]+height), color=1, thickness=2)
        #cv2.line(img, (int(bbox[0]+width*0.5), int(bbox[1] + height)) , (int(bbox[0]+width*0.5) , int(bbox[1] )) ,color=2,thickness=2 )
        #plt.imshow(img)
        #plt.show()

        #-------------------------------------------------#

        img = cv2.imread(path)

        source_polygon = polygons[0]
        try:
            destination_polygon_one = np.array([[x+50,y-100] for x,y in source_polygon  ],dtype=np.int32)
            new_img = mod_image(img,source_polygon,destination_polygon_one)
        except:
            try:
                destination_polygon_one = np.array([[x+150,y+100] for x,y in source_polygon  ],dtype=np.int32)
                new_img = mod_image(img,source_polygon,destination_polygon_one)
            except:
                multi=False


    if multi:
        source_polygon = polygons[1]
        try:
            destination_polygon_two = np.array([[x+200,y-100] for x,y in source_polygon  ],dtype=np.int32)
            new_img2 = mod_image(new_img,source_polygon,destination_polygon_two)
        except:
            try:
                destination_polygon_two = np.array([[x-100,y-50] for x,y in source_polygon  ],dtype=np.int32)
                new_img2 = mod_image(new_img,source_polygon,destination_polygon_two)
            except:
                try:
                    destination_polygon_two = np.array([[x-10,y-10] for x,y in source_polygon  ],dtype=np.int32)
                    new_img2 = mod_image(new_img,source_polygon,destination_polygon_two)
                except:
                    try:
                        destination_polygon_two = np.array([[x+10,y+10] for x,y in source_polygon  ],dtype=np.int32)
                        new_img2 = mod_image(new_img,source_polygon,destination_polygon_two)
                    except:
                        multi = False


        #fig,ax = plt.subplots(1,2,figsize=(15,10))
        #ax[0].imshow(img)
        #ax[1].imshow(new_img2) 
        #plt.show()

        # new_img2 is new image - save it somewhere
        # need new bounding boxes, with multilabels: source_polygon and destination_polygon
        # schema == [ True/False multilabel, class, integer for target counter, sub-integer for segment count]

    df_now = label_target_df.iloc[i]

    annotation_dic = {} # # "id": 9465, "image_id": 9465, "category_id": 4, "segmentation": [[88.60215053763437, 677.7956989247311]], "bbox": [80.0, 677.7956989247311, 80.64516129032256, 140.86021505376345], "iscrowd": 0, "area": 10283.0}
        
    
    # raw target
    annotation_dic["id"] = id
    annotation_dic["image_id"] = df_now["image_id"]
    annotation_dic["category_id"] = df_now["category_id"]
    annotation_dic["segmentation"] = df_now["segmentation"]
    annotation_dic["bbox"] = df_now["bbox"]
    annotation_dic["multilabel"] = 0
    annotation_dic["multi_id"] = 0
    annotation_dic["iscrowd"] = df_now["iscrowd"]
    annotation_dic["area"] = df_now["area"]

    # copy raw image to directory
    os.system(f"ln -s {path} {out_frag_img_dir}images/{label_target_df['img_name'].values[i]}")

    annotations.append(annotation_dic)

    id+=1

    #-------------------------------------------------#

    if multi:

        # copy new image and add new name
        
        cv2.imwrite(f"{out_frag_img_dir}images/xray_{max_image_id}.png", new_img2)

        images.append({"height": new_img2.shape[0], "width": new_img.shape[1], "id": max_image_id, "file_name": f"xray_{max_image_id}.png"})

        annotation_dic = {}
        # first fragmented target
        annotation_dic["id"] = id
        annotation_dic["image_id"] = max_image_id
        annotation_dic["category_id"] = df_now["category_id"]
        annotation_dic["segmentation"] = destination_polygon_one

        min_coords = np.min(destination_polygon_one, axis=0)
        max_coords = np.max(destination_polygon_one, axis=0)

        min_x, min_y = min_coords[0], min_coords[1]
        max_x, max_y = max_coords[0], max_coords[1]

        annotation_dic["bbox"] = [min_x, min_y, max_x, max_y]
        annotation_dic["multilabel"] = 1
        annotation_dic["multi_id"] = 0
        annotation_dic["iscrowd"] = df_now["iscrowd"]
        annotation_dic["area"] = df_now["area"]

        annotations.append(annotation_dic)

        id+=1


        #-------------------------------------------------#


        # second fragmented target
        annotation_dic = {}
        # first fragmented target
        annotation_dic["id"] = id
        annotation_dic["image_id"] = max_image_id
        annotation_dic["category_id"] = df_now["category_id"]
        annotation_dic["segmentation"] = destination_polygon_two

        min_coords = np.min(destination_polygon_two, axis=0)
        max_coords = np.max(destination_polygon_two, axis=0)

        min_x, min_y = min_coords[0], min_coords[1]
        max_x, max_y = max_coords[0], max_coords[1]

        annotation_dic["bbox"] = [min_x, min_y, max_x, max_y]
        annotation_dic["multilabel"] = 1
        annotation_dic["multi_id"] = 1
        annotation_dic["iscrowd"] = df_now["iscrowd"]
        annotation_dic["area"] = df_now["area"]

        annotations.append(annotation_dic)

        id+=1

        max_image_id+=1
    

  0%|          | 0/1332 [00:00<?, ?it/s]

ln: failed to create symbolic link '/teamspace/studios/this_studio/data/fragmented/images/xray_00609.png': File exists
  0%|          | 1/1332 [00:00<02:46,  8.01it/s]ln: failed to create symbolic link '/teamspace/studios/this_studio/data/fragmented/images/xray_00610.png': File exists
ln: failed to create symbolic link '/teamspace/studios/this_studio/data/fragmented/images/xray_00611.png': File exists
  0%|          | 3/1332 [00:00<02:06, 10.54it/s]ln: failed to create symbolic link '/teamspace/studios/this_studio/data/fragmented/images/xray_00612.png': File exists
ln: failed to create symbolic link '/teamspace/studios/this_studio/data/fragmented/images/xray_00613.png': File exists
  0%|          | 5/1332 [00:00<01:44, 12.71it/s]ln: failed to create symbolic link '/teamspace/studios/this_studio/data/fragmented/images/xray_00614.png': File exists
ln: failed to create symbolic link '/teamspace/studios/this_studio/data/fragmented/images/xray_00615.png': File exists
  1%|          | 7/1332

In [12]:
images = images + labels['images']
categories = labels['categories']

In [13]:
len(images)

77944

In [14]:
final_dic = {}

final_dic['info'] = labels['info']

print(labels.keys())

dict_keys(['info', 'license', 'annotations', 'categories', 'images'])
